# Imports

In [ ]:
from ngsolve.meshes import *
from ngsolve import *
from netgen.geom2d import *
from draw import Draw

# Mesh generation of domain $(0,1)^2$
Structured quadrilateral or unstructured mesh:

In [ ]:
unstructured = False

In [ ]:
if unstructured:
    periodic = SplineGeometry()
    pnts = [ (0,0), (1,0), (1,1), (0,1) ]
    pnums = [periodic.AppendPoint(*p) for p in pnts]
    periodic.Append ( ["line", pnums[0], pnums[1]],bc="bottom")
    lright = periodic.Append ( ["line", pnums[1], pnums[2]], bc="periodic")
    periodic.Append ( ["line", pnums[2], pnums[3]], bc="top")
    periodic.Append ( ["line", pnums[0], pnums[3]], leftdomain=0, rightdomain=1, copy=lright, bc="periodic")
    mesh = Mesh(periodic.GenerateMesh(maxh=1/12.8))
else:
    mesh = MakeStructured2DMesh(nx=16, ny=4, periodic_x=True)

# DG FESpace

In [ ]:
order = 2
fes = L2(mesh, order=order, dgjumps=True, all_dofs_together=True)
u, v = fes.TnT()

# Physical parameters 
$$
      - \operatorname{div}( \boldsymbol{\epsilon} \nabla u) + \mathbf{w}^\ast \cdot \nabla u = 0 \quad \text{in } Q.
$$
  where $\boldsymbol{\epsilon} = \left( \begin{array}{cc} \varepsilon & 0 \\ 0 & 0 \end{array} \right)$, $\mathbf{w}^\ast(x,t) = \left( \begin{array}{c} \mathbf{w}(x,t) \\ 1 \end{array} \right)$, $\nabla^\ast=\left( \begin{array}{c} \partial_x \\ \partial_t \end{array} \right)$ and $n_Q$ the outer normal on $Q$. 

In [ ]:
eps = 0.005
b = CoefficientFunction((cos(4*pi*y), 1))
ubnd = exp(-40*(x-0.5)**2)

# Discretization parameters: 
$\lambda$: Symmetric interior penalty parameter

In [ ]:
lambd = 40
h = specialcf.mesh_size
n = specialcf.normal(mesh.dim)

# Diffusion discretization:
Symmetric interior penalty acting only in spatial direction. Jump and average terms as usual except for the selection of the spatial component `[0]` and the weighting with `n[0]`:

In [ ]:
space_jump_u = (u-u.Other())*(n[0])
space_jump_v = (v-v.Other())*(n[0])
space_mean_dudn = 0.5*n[0] * (grad(u)[0]+grad(u.Other())[0])
space_mean_dvdn = 0.5*n[0] * (grad(v)[0]+grad(v.Other())[0])

This yields the diffusion operator:

In [ ]:
space_diffusion = grad(u)[0]*grad(v)[0] * dx \
    + lambd*order**2/h*space_jump_u*space_jump_v*dx(skeleton=True) \
    + (-space_mean_dudn*space_jump_v-space_mean_dvdn*space_jump_u)*dx(skeleton=True)

# Space-time convection
The convection term is the usual one, but interpreted as a space-time operator:

In [ ]:
space_time_convection = -b * u * grad(v)*dx \
    + b*n*IfPos(b*n, u, u.Other()) * (v-v.Other()) * dx(skeleton=True) \
    + b*n*u*v * ds(definedon=mesh.Boundaries("top"),skeleton=True)

# Setup of linear system:

In [ ]:
a = BilinearForm(fes)
a += eps * space_diffusion + space_time_convection
a.Assemble()

In [ ]:
f = LinearForm(fes)
f += ubnd * v * ds(definedon=mesh.Boundaries("bottom"),skeleton=True)
f.Assemble()

# Solution of linear system and visualization

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs(), inverse="umfpack") * f.vec
Draw(0.4*gfu, mesh, "u")

# Display of the sparsity pattern:

In [ ]:
import scipy.sparse as sp
import matplotlib.pyplot as plt

plt.spy(sp.csr_matrix(a.mat.CSR()),precision=0,markersize=64*5/fes.ndof)
plt.show()